In [25]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

extracting data

In [8]:
!pip install PyPDF2


     -------------------------------------- 232.6/232.6 kB 2.0 MB/s eta 0:00:00


In [26]:
import PyPDF2

def read_file(file_path):
    with open(file_path, 'rb') as file:
        if file_path.endswith(".pdf"):
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
            return text
        elif file_path.endswith(".txt"):
            return file.read().decode('utf-8')
        else:
            raise ValueError("Unsupported file type")

data = read_file('C:\\Users\\DELL\\OneDrive\\Desktop (1)\\kfc chatbot\\data\\data.txt')

In [ ]:
data

#pine cone vector database

data chunking

In [28]:
def text_split(extracted_data, chunk_size=500, chunk_overlap=20):
    text_chunks = []
    start = 0
    while start < len(extracted_data):
        end = min(start + chunk_size, len(extracted_data))
        text_chunks.append(extracted_data[start:end])
        start += chunk_size - chunk_overlap

    return text_chunks

In [29]:
datachunks = text_split(data)

In [ ]:
datachunks[0]

In [47]:
len(datachunks)

8

In [ ]:
for chunk in datachunks:
    print(chunk) 

loading data into vector database

initilizing vector database

In [33]:
import os

In [32]:
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
pinecone_key=os.getenv('pineconekey')

In [35]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(pinecone_key)

In [36]:
pc

In [37]:
existing_indexes = pc.list_indexes()

In [46]:
index_name = "kfc-chatbot"

embedding

In [50]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [52]:
from dotenv import load_dotenv
load_dotenv()

True

In [53]:
openaikey=os.getenv('openaikey')

In [54]:
os.environ['openai_api_key']=openaikey

In [200]:
embedding=emb=OpenAIEmbeddings()

In [69]:
from langchain.vectorstores import Pinecone as LangchainPinecone

In [87]:
from openai import OpenAI
client = OpenAI()

def embed(docs: list[str]) -> list[list[float]]:
    res = client.embeddings.create(
        input=docs,
        model="text-embedding-3-small"
    )
    doc_embeds = [r.embedding for r in res.data]
    return doc_embeds

texts = datachunks
doc_embeds = embed(texts)
print("Embeddings:", doc_embeds)

Embeddings: [[-0.02501089498400688, -0.020139766857028008, 0.04391433671116829, 0.024232009425759315, -0.006163090001791716, -0.026704663410782814, -0.016863500699400902, 0.04584300518035889, -0.03694145008921623, 0.07966890931129456, 0.012530174106359482, -0.03256485238671303, 0.028212981298565865, -0.0005335523746907711, -0.006178544368594885, -0.0015353636117652059, 0.023280037567019463, 0.027693724259734154, -0.0507388599216938, -0.0019456696463748813, 0.03308410942554474, -0.04814257472753525, -0.04186203330755234, 0.003987154457718134, 0.013203972019255161, 0.07299274951219559, -0.05642596259713173, -0.029993293806910515, -0.04490339756011963, -0.018136916682124138, 0.03397426754236221, -0.02394765429198742, -0.06176689639687538, 0.018359456211328506, 0.007238694466650486, -0.02149972692131996, 0.0197441428899765, 0.04549683257937431, 0.01429194025695324, 0.00294554908759892, -0.029053684324026108, -0.03385063260793686, -0.03496332839131355, 0.019682325422763824, 0.02764427103102

In [88]:
len(doc_embeds)

8

In [ ]:
vectors = []
for idx, (text, embedding) in enumerate(zip(datachunks, doc_embeds)):
    vectors.append({
        "id": str(idx+1),
        "values": embedding,
        "metadata": {'text': text}
    })

print("Vectors:", vectors)


In [101]:
vectors

[{'id': '1',
  'values': [-0.02501089498400688,
   -0.020139766857028008,
   0.04391433671116829,
   0.024232009425759315,
   -0.006163090001791716,
   -0.026704663410782814,
   -0.016863500699400902,
   0.04584300518035889,
   -0.03694145008921623,
   0.07966890931129456,
   0.012530174106359482,
   -0.03256485238671303,
   0.028212981298565865,
   -0.0005335523746907711,
   -0.006178544368594885,
   -0.0015353636117652059,
   0.023280037567019463,
   0.027693724259734154,
   -0.0507388599216938,
   -0.0019456696463748813,
   0.03308410942554474,
   -0.04814257472753525,
   -0.04186203330755234,
   0.003987154457718134,
   0.013203972019255161,
   0.07299274951219559,
   -0.05642596259713173,
   -0.029993293806910515,
   -0.04490339756011963,
   -0.018136916682124138,
   0.03397426754236221,
   -0.02394765429198742,
   -0.06176689639687538,
   0.018359456211328506,
   0.007238694466650486,
   -0.02149972692131996,
   0.0197441428899765,
   0.04549683257937431,
   0.01429194025695324,


In [102]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(pinecone_key)

In [103]:
index=pc.Index('kfc-chatbot')

In [104]:
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 8}

In [152]:
text_query = "hi"
query_vector = embed([text_query])[0]

In [153]:
len(query_vector)

1536

In [ ]:
results = index.query(
    top_k=1, 
    vector=query_vector,
    namespace="ns1"
)
print("Query Results:", results)


In [ ]:
ids_to_fetch = [match['id'] for match in results['matches']]
fetched_data = index.fetch(ids=ids_to_fetch, namespace="ns1")
print("Fetched Data:", fetched_data)

In [ ]:
for vector_id, vector_data in fetched_data.get('vectors', {}).items():
    metadata = vector_data.get('metadata', {})
    text = metadata.get('text', 'No text found')
    print(f"Text: {text}")

In [125]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below,just to the point answer and nothing else.
Helpful answer:
"""

In [210]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [177]:
from langchain_openai import ChatOpenAI

In [178]:
llm=ChatOpenAI()

In [146]:
from langchain.chains import RetrievalQA
from langchain.chains import AnalyzeDocumentChain

In [211]:
from langchain.vectorstores import Pinecone as LangPinecone

In [171]:
index = pc.Index(index_name)
index

In [212]:
vectorstore = LangPinecone(index=index, embedding=embedding, text_key="text", namespace="ns1")

In [187]:
vector_store = Pinecone(index=index, embedding=embedding, text_key="text")
retriever = vector_store.as_retriever()
chain_type_kwargs={"prompt": PROMPT}

In [217]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
    output_key="result"
)

In [225]:
result = qa({"query": 'give kfc deals for family?'})
print("Result:", result['result'])
# print("Source Documents:", result['source_documents'])

Result: The KFC Family Feast deal includes 8 pieces of chicken, 4 biscuits, 2 large sides, and 1 large drink for Rs 2,499+Tax.


In [222]:
print(result)

{'query': 'website link', 'result': 'Order Online: www.order.kfc.pk', 'source_documents': [Document(page_content='ayment options are accepted.\r\n\r\n## Links\r\n- *Order Online*: www.order.kfc.pk\r\n- *Customer Reviews*: www.reviews.kfc.pk'), Document(page_content="*: Classic fried chicken with a secret blend of 11 herbs and spices.\r\n  - *Extra Crispy*: Double-breaded fried chicken for extra crunch.\r\n  - *Kentucky Grilled Chicken*: Marinated and grilled chicken for a healthier option.\r\n  - *Chicken Tenders*: Crispy strips of chicken breast.\r\n\r\n- *Sandwiches*:\r\n  - *Zinger Burger*: Spicy fried chicken fillet with lettuce and mayo in a bun.\r\n  - *Twister Wrap*: Fried chicken strips with lettuce, tomatoes, and pepper mayo in a tortilla wrap.\r\n  - *Colonel's C")]}
